# Github repo

clone github repository

In [1]:
!git clone https: // github.com/thomasmaliappis/context-group-detection.git

Cloning into 'context-group-detection'...
remote: Enumerating objects: 814, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 814 (delta 78), reused 84 (delta 39), pack-reused 688
Receiving objects: 100% (814/814), 1.55 MiB | 2.46 MiB/s, done.
Resolving deltas: 100% (465/465), done.


move in repository folder

In [2]:
%cd context-group-detection

/content/context-group-detection


change branch

In [3]:
!git checkout dante

Branch 'dante' set up to track remote branch 'dante' from 'origin'.
Switched to a new branch 'dante'


# Drive

mount drive

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Requirements
install missing packages

In [5]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 8.6 MB/s eta 0:00:00


In [6]:
!pip install pykalman

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48442 sha256=f3166642da2721d07fdc9599f3b1e26753a2009fb55b29840ac13e68f81d46f5
  Stored in directory: /root/.cache/pip/wheels/76/33/ef/5f332226e13a5089c6dd4b01cc2bcb59491d18f955fa2d3807
Successfully built pykalman


# Imports

In [15]:
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, TensorBoard

from models.utils import load_data, save_model_data, build_model, ValLoss

# Functions

class to handle arguments instead of argument parser

In [9]:
class Args:
    def __init__(self, dataset, dataset_path, epochs, features, agents, frames, batch_size, reg, dropout, learning_rate,
                 gmitre_calc):
        self.dataset = dataset
        self.dataset_path = dataset_path
        self.epochs = epochs
        self.features = features
        self.agents = agents
        self.frames = frames
        self.batch_size = batch_size
        self.reg = reg
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.gmitre_calc = gmitre_calc


# Main

set argument constants

In [10]:
args = Args(
    dataset='eth',
    dataset_path='./datasets/ETH/seq_eth',
    epochs=50,
    features=4,
    agents=10,
    frames=10,
    batch_size=1024,
    reg=0.0000001,
    dropout=0.35,
    learning_rate=0.0001,
    gmitre_calc=True
)

initialise variables

In [11]:
tf.random.set_seed(0)
np.random.seed(0)

global_filters = [64, 128, 512]
individual_filters = [16, 64, 128]
combined_filters = [256, 64]

load data

In [13]:
train, test, val = load_data('/content/drive/My Drive/datasets/{}_1_{}'.format(args.dataset, args.agents))

create and train model

In [ ]:
_, _, max_people, d = train[0][0].shape

# build model
model = build_model(args.reg, args.dropout, max_people, d, global_filters, individual_filters, combined_filters)

# train model
tensorboard = TensorBoard(log_dir='./logs')
early_stop = EarlyStopping(monitor='val_loss', patience=20)
history = ValLoss(val, args.dataset, args.dataset_path)

model.fit(train[0], train[1], epochs=args.epochs, batch_size=args.batch_size,
          validation_data=(val[0], val[1]), callbacks=[tensorboard, history, early_stop])

Epoch 1/50
 5/78 [>.............................] - ETA: 1s - loss: 0.9363 - mse: 0.3172  

663/663 [==============================] - 3s 4ms/step


keep track of evaluation metrics

In [ ]:
save_model_data(args.dataset, args.reg, args.dropout, history, test, gmitre_calc=args.gmitre_calc)

saving model to models/eth/pair_predictions_2
1913/1913 [==============================] - 17s 9ms/step
saved best val model as /best_val_model.h5
